In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [6]:
subject = 'Rondônia - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [7]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_RO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data = data.drop('Rondônia - Produção de Cimento (t)',axis=1)
data

,Ano,Rondônia - IDH Longevidade,Rondônia - Desemprego,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH,Rondônia - IDH Renda,Contratação Comercial (Bi R$),...,Taxa Selic (%),Rondônia - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rondônia - Consumo de Cimento (t)
0,2008,0.751494,7.979888,2.208447e+07,1.637885e+06,11.689351,1.870941e+07,0.698932,0.693174,5.483335e+08,...,11.760983,0.655238,22.808561,1006.894941,19176.848685,26.497969,21.685776,75.469241,5.070682,390.788000
1,2009,0.752810,8.008176,2.226473e+07,1.643078e+06,11.684640,1.873529e+07,0.700638,0.693109,5.302764e+08,...,9.472585,0.659243,19.588422,1006.775008,19127.730770,26.528813,21.707671,75.437690,4.902049,452.325000
2,2010,0.753706,7.971248,2.285838e+07,1.640521e+06,11.724602,1.891099e+07,0.702254,0.693063,5.046425e+08,...,9.332485,0.663312,18.596388,1006.795758,14826.071459,26.434620,21.583453,75.296199,4.742629,921.425000
3,2011,0.754192,7.956614,2.338128e+07,1.618454e+06,11.764179,1.903017e+07,0.703786,0.693036,4.854622e+08,...,10.985689,0.667449,18.419365,1007.036992,8438.763610,26.447727,21.511515,74.910155,4.560734,1021.570000
4,2012,0.754233,6.546489,2.383098e+07,1.560372e+06,11.757682,1.910530e+07,0.705401,0.692995,4.867661e+08,...,8.159014,0.672215,19.487867,1007.195877,4928.830653,26.509272,21.598374,75.028438,4.420622,933.501000
5,2013,0.753919,6.917221,2.422272e+07,1.518707e+06,11.741719,1.916060e+07,0.706351,0.693078,5.225848e+08,...,7.913561,0.675149,21.002459,1007.408812,3448.198287,26.513099,21.752006,75.695263,4.276194,826.670000
6,2014,0.753246,7.287260,2.458709e+07,1.494999e+06,11.767208,1.922796e+07,0.707964,0.693001,5.676413e+08,...,10.350442,0.680412,20.055465,1007.673651,2597.909114,26.561558,21.940898,76.327127,4.207147,614.394714
7,2015,0.752267,7.654493,2.486993e+07,1.466317e+06,11.788091,1.927573e+07,0.709294,0.693168,5.179392e+08,...,12.454398,0.684863,18.767358,1007.697489,2136.164001,26.644505,22.123957,76.795244,4.212726,510.273771
8,2016,0.759242,7.643673,2.507534e+07,1.450033e+06,11.811765,1.931637e+07,0.714910,0.693113,4.524144e+08,...,13.111908,0.695058,17.848044,1007.584830,1863.004107,26.594749,22.202418,77.390294,4.183474,410.580514
9,2017,0.758083,7.282097,2.517074e+07,1.454651e+06,11.835195,1.934371e+07,0.714902,0.693760,4.213689e+08,...,9.479812,0.695427,17.563275,1007.547716,1682.028677,26.577385,22.310278,77.982455,4.075241,406.814000


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Rondônia - IDH Longevidade,Rondônia - Desemprego,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH,Rondônia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rondônia - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,0.751494,7.979888,2.208447e+07,1.637885e+06,11.689351,1.870941e+07,0.698932,0.693174,5.483335e+08,2.546644e+09,...,2.868108,11.760983,0.655238,22.808561,1006.894941,19176.848685,26.497969,21.685776,75.469241,5.070682
1,0.752810,8.008176,2.226473e+07,1.643078e+06,11.684640,1.873529e+07,0.700638,0.693109,5.302764e+08,2.679945e+09,...,2.305885,9.472585,0.659243,19.588422,1006.775008,19127.730770,26.528813,21.707671,75.437690,4.902049
2,0.753706,7.971248,2.285838e+07,1.640521e+06,11.724602,1.891099e+07,0.702254,0.693063,5.046425e+08,2.891930e+09,...,1.958246,9.332485,0.663312,18.596388,1006.795758,14826.071459,26.434620,21.583453,75.296199,4.742629
3,0.754192,7.956614,2.338128e+07,1.618454e+06,11.764179,1.903017e+07,0.703786,0.693036,4.854622e+08,3.091988e+09,...,2.079088,10.985689,0.667449,18.419365,1007.036992,8438.763610,26.447727,21.511515,74.910155,4.560734
4,0.754233,6.546489,2.383098e+07,1.560372e+06,11.757682,1.910530e+07,0.705401,0.692995,4.867661e+08,3.189506e+09,...,2.183282,8.159014,0.672215,19.487867,1007.195877,4928.830653,26.509272,21.598374,75.028438,4.420622
5,0.753919,6.917221,2.422272e+07,1.518707e+06,11.741719,1.916060e+07,0.706351,0.693078,5.225848e+08,3.093872e+09,...,2.403445,7.913561,0.675149,21.002459,1007.408812,3448.198287,26.513099,21.752006,75.695263,4.276194
6,0.753246,7.287260,2.458709e+07,1.494999e+06,11.767208,1.922796e+07,0.707964,0.693001,5.676413e+08,2.779340e+09,...,2.748397,10.350442,0.680412,20.055465,1007.673651,2597.909114,26.561558,21.940898,76.327127,4.207147
7,0.752267,7.654493,2.486993e+07,1.466317e+06,11.788091,1.927573e+07,0.709294,0.693168,5.179392e+08,2.561731e+09,...,2.501529,12.454398,0.684863,18.767358,1007.697489,2136.164001,26.644505,22.123957,76.795244,4.212726
8,0.759242,7.643673,2.507534e+07,1.450033e+06,11.811765,1.931637e+07,0.714910,0.693113,4.524144e+08,2.516859e+09,...,2.073464,13.111908,0.695058,17.848044,1007.584830,1863.004107,26.594749,22.202418,77.390294,4.183474
9,0.758083,7.282097,2.517074e+07,1.454651e+06,11.835195,1.934371e+07,0.714902,0.693760,4.213689e+08,2.497710e+09,...,2.177386,9.479812,0.695427,17.563275,1007.547716,1682.028677,26.577385,22.310278,77.982455,4.075241


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      452.325000
1      921.425000
2     1021.570000
3      933.501000
4      826.670000
5      614.394714
6      510.273771
7      410.580514
8      406.814000
9      376.138000
10     399.441000
11     488.194000
12     502.824000
13     501.142000
14            NaN
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Rondônia - IDH Longevidade,Rondônia - Desemprego,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH,Rondônia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rondônia - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.366933,1.128601,-1.793068,1.343669,-1.602309,-1.897543,-1.597405,0.095187,1.204752,-0.990234,...,2.026352,0.984445,-1.605695,2.457245,-1.212494,1.862951,-0.705394,-0.762192,-0.756764,1.972247
1,-0.774200,1.188885,-1.627168,1.412024,-1.699639,-1.776559,-1.284617,-0.087133,0.820657,-0.546688,...,0.085558,-0.051710,-1.324921,0.144226,-1.582145,1.855564,-0.347758,-0.699954,-0.781667,1.489960
2,-0.370197,1.110188,-1.080790,1.378366,-0.873963,-0.955015,-0.988265,-0.215819,0.275392,0.158672,...,-1.114490,-0.115145,-1.039610,-0.568350,-1.518190,1.208644,-1.439934,-1.053053,-0.893344,1.034020
3,-0.151306,1.079003,-0.599524,1.087914,-0.056217,-0.397801,-0.707210,-0.291249,-0.132596,0.824346,...,-0.697344,0.633403,-0.749553,-0.695505,-0.774675,0.248067,-1.287958,-1.257539,-1.198046,0.513803
4,-0.132584,-1.926102,-0.185632,0.323418,-0.190465,-0.046510,-0.410992,-0.406659,-0.104860,1.148829,...,-0.337667,-0.646476,-0.415389,0.071997,-0.284967,-0.279787,-0.574331,-1.010638,-1.104686,0.113084
5,-0.274035,-1.136039,0.174914,-0.224995,-0.520296,0.212086,-0.236740,-0.174695,0.657046,0.830615,...,0.422337,-0.757614,-0.209634,1.159926,0.371328,-0.502457,-0.529965,-0.573928,-0.578367,-0.299977
6,-0.577490,-0.347451,0.510268,-0.537052,0.006355,0.527031,0.059020,-0.389665,1.615453,-0.215960,...,1.613108,0.345772,0.159340,0.479702,1.187601,-0.630330,0.031926,-0.036991,-0.079642,-0.497452
7,-1.018893,0.435155,0.770583,-0.914581,0.437846,0.750387,0.303008,0.076848,0.558230,-0.940033,...,0.760923,1.298413,0.471418,-0.445543,1.261073,-0.699772,0.993697,0.483365,0.289839,-0.481495
8,2.124681,0.412096,0.959640,-1.128914,0.927003,0.940414,1.333009,-0.076826,-0.835563,-1.089341,...,-0.716758,1.596124,1.186272,-1.105884,0.913840,-0.740852,0.416771,0.706397,0.759507,-0.565156
9,1.602027,-0.358454,1.047447,-1.068126,1.411104,1.068244,1.331508,1.732167,-1.495939,-1.153056,...,-0.358019,-0.048438,1.212138,-1.310433,0.799449,-0.768068,0.215438,1.012997,1.226895,-0.874702


In [11]:
reshaped_train_input = dfToInputRNN(train_input)

In [12]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.11079346, -0.82973435,  0.81755237, -0.70869537,
          0.86489319,  0.68245254,  0.93691966, -1.02376433,
         -1.0453709 ,  1.21859064, -0.99976018,  0.79428371,
          1.15575749, -0.52559858,  1.30260559, -0.55253046,
         -0.25130359,  1.38787176,  1.47154451, -1.80649616,
          1.09296291,  0.94281138,  0.52836084,  0.61258103,
          0.87656257,  1.47801536,  1.71982086,  1.28221181,
          1.39086472,  1.58209844,  1.08872091,  0.55212962,
         -1.81398118,  1.44113674,  1.11191094,  1.30470835,
          0.86390516, -1.67365968, -1.70738939, -1.55813196,
         -0.48498301, -0.75714639,  0.40770893, -0.27394734,
          2.02323219,  1.2908951 ,  1.70465513,  1.81148084,
          1.39968653, -1.36191442,  1.28024593,  1.11054795,
          0.12662799,  1.40414112,  1.40468999,  1.45643959,
          1.58509336,  1.32283157,  1.30222047,  1.55138602,
          1.91801602, -0.77489784,  2.32098884, -1.1942541 ,
         -1.17081042,  1

In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0      452.325000
1      921.425000
2     1021.570000
3      933.501000
4      826.670000
5      614.394714
6      510.273771
7      410.580514
8      406.814000
9      376.138000
10     399.441000
11     488.194000
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [14]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    488.194
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Rondônia - IDH Longevidade,Rondônia - Desemprego,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH,Rondônia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rondônia - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-0.970117,-1.191413,0.552069,-0.469187,0.356059,0.388444,0.645784,-2.881361,1.108405,2.635823,...,-1.482779,-2.260447,0.964061,-2.07127,-0.395184,-0.716176,2.608857,1.699552,1.207817,-1.029715


In [16]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[-0.97011657, -1.1914133 ,  0.55206853, -0.46918735,
          0.35605886,  0.38844425,  0.64578366, -2.88136138,
          1.10840491,  2.63582309, -0.18995911,  0.72640489,
          2.66833308, -0.032632  ,  0.98802435,  0.24980494,
         -0.08955808,  1.85434394,  2.02706568, -2.21150703,
          1.42938591,  0.85132668,  0.14092386,  0.30902324,
          1.00267904,  2.27970137,  1.11272593,  0.61007902,
          0.89064538,  1.42231438,  1.02650562,  1.47127593,
         -1.53201473,  1.73503858,  1.66610613,  1.0456894 ,
          0.55289259, -1.81216593, -2.10342296, -2.55377478,
          0.41940628, -1.75756445,  0.241023  , -0.98659345,
          1.52184916,  1.01398607,  1.49290967,  1.40764092,
          1.87930411, -2.51892072,  2.83356214,  1.58620924,
         -0.02524128,  1.54231542,  1.5176119 ,  1.7355269 ,
          1.87941982,  1.54198129,  1.51276712,  1.83795594,
          2.62210693,  1.04993174,  0.96476493, -1.48277861,
         -2.26044738,  0

In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    502.824
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [18]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=750, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True)
    
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh', return_sequences=True)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [20]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(200)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [22]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2222318583, 1114936464, 1788642522, 2013630944, 243933120, 2675810934, 1448901479, 2661798423, 3301596469, 2340489961, 673750834, 2064602504, 1275321839, 1358885789, 2144928210, 887569525, 1223099548, 2727799396, 2022062272, 206894034, 1858024344, 1258934096, 911566377, 3475731864, 3512291895, 2612026113, 3531157982, 1161144199, 2607478330, 848603551, 1102961426, 3838090894, 551206288, 3834055974, 230839290, 528244783, 1616252330, 2076921998, 2038300636, 3008240302, 3975129660, 373167768, 3960362725, 4281687526, 2818970881, 4245589051, 270792791, 283157207, 1312808547, 1221860298, 227762611, 2131752789, 1421383388, 3604892460, 4017290685, 2088357327, 3553364871, 2249539099, 1926439983, 2527147075, 2216991858, 3362457431, 2770826312, 3906470157, 2138437922, 27042315, 1244781367, 1247843410, 441354192, 3318339807, 2571763559, 3228109168, 3722196920, 1310627666, 1580976652, 948131376, 3827824981, 3331393769, 3321329439, 55551320, 1262729486, 1956205090, 2744310582, 2730917667, 3053319390

loss: 7985.669921875


Step: 75 ___________________________________________
loss: 3562.608642578125


Step: 76 ___________________________________________
loss: 6603.53759765625


Step: 77 ___________________________________________
loss: 6514.39892578125


Step: 78 ___________________________________________
loss: 589.9190063476562


Step: 79 ___________________________________________
loss: 792.33935546875


Step: 80 ___________________________________________
loss: 6985.63427734375


Step: 81 ___________________________________________
loss: 6315.80322265625


Step: 82 ___________________________________________
loss: 4240.25146484375


Step: 83 ___________________________________________
loss: 7173.91552734375


Step: 84 ___________________________________________
loss: 6897.17138671875


Step: 85 ___________________________________________
loss: 1779.0126953125


Step: 86 ___________________________________________
loss: 7049.74609375


Step: 87 ___________________________________

loss: 3602.910888671875


Step: 179 ___________________________________________
loss: 6832.86572265625


Step: 180 ___________________________________________
loss: 4169.38623046875


Step: 181 ___________________________________________
loss: 5497.80615234375


Step: 182 ___________________________________________
loss: 3421.729736328125


Step: 183 ___________________________________________
loss: 8278.2998046875


Step: 184 ___________________________________________
loss: 846.271484375


Step: 185 ___________________________________________
loss: 820.1076049804688


Step: 186 ___________________________________________
loss: 1908.8443603515625


Step: 187 ___________________________________________
loss: 507.183349609375


Step: 188 ___________________________________________
loss: 4165.568359375


Step: 189 ___________________________________________
loss: 2598.391357421875


Step: 190 ___________________________________________
loss: 2522.392333984375


Step: 191 ________________

In [23]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 430268.2500
Epoch 2/10000
1/1 [==============================] - 0s 8ms/step - loss: 421347.1250
Epoch 3/10000
1/1 [==============================] - 0s 7ms/step - loss: 394333.0312
Epoch 4/10000
1/1 [==============================] - 0s 7ms/step - loss: 222623.7969
Epoch 5/10000
1/1 [==============================] - 0s 7ms/step - loss: 42556.4062
Epoch 6/10000
1/1 [==============================] - 0s 8ms/step - loss: 36183.6758
Epoch 7/10000
1/1 [==============================] - 0s 9ms/step - loss: 29945.7207
Epoch 8/10000
1/1 [==============================] - 0s 8ms/step - loss: 23051.9922
Epoch 9/10000
1/1 [==============================] - 0s 5ms/step - loss: 25002.1660
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 25636.1426
Epoch 11/10000
1/1 [==============================] - 0s 8ms/step - loss: 21728.4922
Epoch 12/10000
1/1 [==============================] - 0s 8ms/step - los

1/1 [==============================] - 0s 6ms/step - loss: 18644.0938
Epoch 98/10000
1/1 [==============================] - 0s 5ms/step - loss: 17134.2598
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 13481.1279
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 19554.7148
Epoch 101/10000
1/1 [==============================] - 0s 5ms/step - loss: 21964.5723
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 20088.6699
Epoch 103/10000
1/1 [==============================] - 0s 5ms/step - loss: 14716.4092
Epoch 104/10000
1/1 [==============================] - 0s 5ms/step - loss: 12223.7725
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 20299.1758
Epoch 106/10000
1/1 [==============================] - 0s 5ms/step - loss: 12092.9570
Epoch 107/10000
1/1 [==============================] - 0s 5ms/step - loss: 11055.2842
Epoch 108/10000
1/1 [==============================] - 0s 5ms/step - los

1/1 [==============================] - 0s 5ms/step - loss: 3426.9717
Epoch 194/10000
1/1 [==============================] - 0s 5ms/step - loss: 2863.3245
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 3001.4678
Epoch 196/10000
1/1 [==============================] - 0s 5ms/step - loss: 3193.9036
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 5243.6304
Epoch 198/10000
1/1 [==============================] - 0s 5ms/step - loss: 3005.5740
Epoch 199/10000
1/1 [==============================] - 0s 5ms/step - loss: 4779.1489
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 2727.9199
Epoch 201/10000
1/1 [==============================] - 0s 5ms/step - loss: 2347.8984
Epoch 202/10000
1/1 [==============================] - 0s 5ms/step - loss: 4170.6357
Epoch 203/10000
1/1 [==============================] - 0s 5ms/step - loss: 6447.0391
Epoch 204/10000
1/1 [==============================] - 0s 5ms/step - loss: 6024.0

1/1 [==============================] - 0s 5ms/step - loss: 4856.6602
Epoch 290/10000
1/1 [==============================] - 0s 5ms/step - loss: 7812.8032
Epoch 291/10000
1/1 [==============================] - 0s 5ms/step - loss: 7477.4077
Epoch 292/10000
1/1 [==============================] - 0s 5ms/step - loss: 4959.4390
Epoch 293/10000
1/1 [==============================] - 0s 5ms/step - loss: 2941.9170
Epoch 294/10000
1/1 [==============================] - 0s 9ms/step - loss: 1225.5201
Epoch 295/10000
1/1 [==============================] - 0s 5ms/step - loss: 4144.8242
Epoch 296/10000
1/1 [==============================] - 0s 5ms/step - loss: 3446.0361
Epoch 297/10000
1/1 [==============================] - 0s 5ms/step - loss: 2992.0315
Epoch 298/10000
1/1 [==============================] - 0s 6ms/step - loss: 4404.9302
Epoch 299/10000
1/1 [==============================] - 0s 6ms/step - loss: 8893.0107
Epoch 300/10000
1/1 [==============================] - 0s 5ms/step - loss: 2790.4

1/1 [==============================] - 0s 4ms/step - loss: 2382.2012
Epoch 386/10000
1/1 [==============================] - 0s 5ms/step - loss: 4156.8862
Epoch 387/10000
1/1 [==============================] - 0s 5ms/step - loss: 5558.3530
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 3915.8347
Epoch 389/10000
1/1 [==============================] - 0s 5ms/step - loss: 2164.0652
Epoch 390/10000
1/1 [==============================] - 0s 5ms/step - loss: 3731.1926
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 4400.4067
Epoch 392/10000
1/1 [==============================] - 0s 4ms/step - loss: 5521.6016
Epoch 393/10000
1/1 [==============================] - 0s 5ms/step - loss: 2933.1042
Epoch 394/10000
1/1 [==============================] - 0s 5ms/step - loss: 5678.2852
Epoch 395/10000
1/1 [==============================] - 0s 4ms/step - loss: 4791.2495
Epoch 396/10000
1/1 [==============================] - 0s 5ms/step - loss: 6864.4

...

1/1 [==============================] - 0s 5ms/step - loss: 1817.6368
Epoch 1722/10000
1/1 [==============================] - 0s 4ms/step - loss: 3708.3967
Epoch 1723/10000
1/1 [==============================] - 0s 5ms/step - loss: 2374.6436
Epoch 1724/10000
1/1 [==============================] - 0s 5ms/step - loss: 3868.8381
Epoch 1725/10000
1/1 [==============================] - 0s 4ms/step - loss: 3063.6790
Epoch 1726/10000
1/1 [==============================] - 0s 6ms/step - loss: 4337.6553
Epoch 1727/10000
1/1 [==============================] - 0s 5ms/step - loss: 2809.9392
Epoch 1728/10000
1/1 [==============================] - 0s 4ms/step - loss: 1594.5516
Epoch 1729/10000
1/1 [==============================] - 0s 5ms/step - loss: 4456.3491
Epoch 1730/10000
1/1 [==============================] - 0s 6ms/step - loss: 5767.6694
Epoch 1731/10000
1/1 [==============================] - 0s 6ms/step - loss: 1989.9979
Epoch 1732/10000
1/1 [==============================] - 0s 6ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 2269.7864
Epoch 1817/10000
1/1 [==============================] - 0s 4ms/step - loss: 3420.6028
Epoch 1818/10000
1/1 [==============================] - 0s 5ms/step - loss: 2563.1292
Epoch 1819/10000
1/1 [==============================] - 0s 5ms/step - loss: 2537.8035
Epoch 1820/10000
1/1 [==============================] - 0s 4ms/step - loss: 2176.6765
Epoch 1821/10000
1/1 [==============================] - 0s 4ms/step - loss: 2989.2695
Epoch 1822/10000
1/1 [==============================] - 0s 4ms/step - loss: 3759.0078
Epoch 1823/10000
1/1 [==============================] - 0s 4ms/step - loss: 2971.1709
Epoch 1824/10000
1/1 [==============================] - 0s 4ms/step - loss: 2651.1042
Epoch 1825/10000
1/1 [==============================] - 0s 4ms/step - loss: 7996.1992
Epoch 1826/10000
1/1 [==============================] - 0s 4ms/step - loss: 1440.4623
Epoch 1827/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 5ms/step - loss: 2453.0359
Epoch 1912/10000
1/1 [==============================] - 0s 6ms/step - loss: 3192.0254
Epoch 1913/10000
1/1 [==============================] - 0s 4ms/step - loss: 1966.3971
Epoch 1914/10000
1/1 [==============================] - 0s 4ms/step - loss: 1129.7695
Epoch 1915/10000
1/1 [==============================] - 0s 4ms/step - loss: 2849.2488
Epoch 1916/10000
1/1 [==============================] - 0s 4ms/step - loss: 1798.0400
Epoch 1917/10000
1/1 [==============================] - 0s 5ms/step - loss: 2402.2842
Epoch 1918/10000
1/1 [==============================] - 0s 4ms/step - loss: 3775.4668
Epoch 1919/10000
1/1 [==============================] - 0s 4ms/step - loss: 5257.4834
Epoch 1920/10000
1/1 [==============================] - 0s 4ms/step - loss: 2193.1799
Epoch 1921/10000
1/1 [==============================] - 0s 4ms/step - loss: 2585.4768
Epoch 1922/10000
1/1 [==============================] - 0s 5ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 2588.7805
Epoch 2007/10000
1/1 [==============================] - 0s 5ms/step - loss: 4661.1118
Epoch 2008/10000
1/1 [==============================] - 0s 4ms/step - loss: 2425.4565
Epoch 2009/10000
1/1 [==============================] - 0s 5ms/step - loss: 5711.7251
Epoch 2010/10000
1/1 [==============================] - 0s 5ms/step - loss: 2410.0989
Epoch 2011/10000
1/1 [==============================] - 0s 4ms/step - loss: 1567.1152
Epoch 2012/10000
1/1 [==============================] - 0s 4ms/step - loss: 3880.5605
Epoch 2013/10000
1/1 [==============================] - 0s 4ms/step - loss: 4639.8965
Epoch 2014/10000
1/1 [==============================] - 0s 4ms/step - loss: 9362.7910
Epoch 2015/10000
1/1 [==============================] - 0s 5ms/step - loss: 3398.9500
Epoch 2016/10000
1/1 [==============================] - 0s 6ms/step - loss: 3296.7434
Epoch 2017/10000
1/1 [==============================] - 0s 5ms/step - l

1/1 [==============================] - 0s 6ms/step - loss: 1793.0524
Epoch 2102/10000
1/1 [==============================] - 0s 7ms/step - loss: 2360.2844
Epoch 2103/10000
1/1 [==============================] - 0s 7ms/step - loss: 4327.8647
Epoch 2104/10000
1/1 [==============================] - 0s 6ms/step - loss: 793.2747
Epoch 2105/10000
1/1 [==============================] - 0s 7ms/step - loss: 4899.9106
Epoch 2106/10000
1/1 [==============================] - 0s 5ms/step - loss: 4521.0249
Epoch 2107/10000
1/1 [==============================] - 0s 6ms/step - loss: 3506.7063
Epoch 2108/10000
1/1 [==============================] - 0s 6ms/step - loss: 4637.2222
Epoch 2109/10000
1/1 [==============================] - 0s 7ms/step - loss: 3845.7539
Epoch 2110/10000
1/1 [==============================] - 0s 6ms/step - loss: 5908.7388
Epoch 2111/10000
1/1 [==============================] - 0s 7ms/step - loss: 3205.3301
Epoch 2112/10000
1/1 [==============================] - 0s 7ms/step - lo

In [24]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 581ms/step
[502.824](test_target) - [[474.34973]](prediction) = 28.474268554687512


In [25]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.056628698221818194

In [26]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [27]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Rondônia - IDH Longevidade,Rondônia - Desemprego,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH,Rondônia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rondônia - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.294335,-0.415281,-0.961963,-1.230898,-0.570897,-0.846639,-1.235610,1.288777,1.148205,-1.122043,...,1.308949,1.412346,-1.221460,1.377871,1.395565,0.719114,0.276338,0.495192,0.906319,1.236047
1,0.153687,1.378391,-0.416778,1.218498,-0.835068,-0.557701,0.022028,-0.140116,0.140893,-0.184474,...,-0.190804,-0.643250,-0.006544,-0.413067,-0.896020,0.695031,1.062967,0.899613,0.487021,-0.022926
2,1.140648,-0.963110,1.378740,0.012400,1.405965,1.404340,1.213582,-1.148661,-1.289098,1.306518,...,-1.118144,-0.769096,1.228004,-0.964804,-0.499545,-1.414145,-1.339305,-1.394805,-1.393340,-1.213121


train_target:


,Rondônia - Consumo de Cimento (t)
0,452.325
1,921.425
2,1021.570


test_input:


...

,Rondônia - IDH Longevidade,Rondônia - Desemprego,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH,Rondônia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rondônia - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-0.970117,-1.191413,0.552069,-0.469187,0.356059,0.388444,0.645784,-2.881361,1.108405,2.635823,...,-1.482779,-2.260447,0.964061,-2.07127,-0.395184,-0.716176,2.608857,1.699552,1.207817,-1.029715


test_target:


,Rondônia - Consumo de Cimento (t)
12,502.824


1/1 [==============================] - 1s 596ms/step
Error: 28.474268554687512




[1022.9430541992188,
 954.5559692382812,
 528.2281494140625,
 582.53759765625,
 505.3045349121094,
 414.6282043457031,
 394.801513671875,
 391.83203125,
 384.046875,
 474.3497314453125]

In [28]:
display(targets)
display(predictions)

[933.501,
 826.67,
 614.3947142857144,
 510.2737714285714,
 410.5805142857144,
 406.814,
 376.138,
 399.441,
 488.194,
 502.824]

[1022.9430541992188,
 954.5559692382812,
 528.2281494140625,
 582.53759765625,
 505.3045349121094,
 414.6282043457031,
 394.801513671875,
 391.83203125,
 384.046875,
 474.3497314453125]

In [29]:
mae = mean_absolute_error(predictions, targets)
mae

63.719051548549125

In [30]:
porcentage = mae/np.mean(targets)
porcentage

0.11651311139171995